In [51]:
# default_exp core

# WorldWideNewsMap

> Fetches the news all around the world

In [52]:
#hide
from nbdev.showdoc import *

In [53]:
class WorldWideNews:
    """
    
    """
    def __init__(self,keywords:list,regions:list,output_language:str='en') -> None:
        self.keywords = keywords
        self.regions = regions
        self.output_language = output_language

    def translate_keywords(self)-> List[KeywordsPerMarket]:
        

    def fetch_articles(self):


IndentationError: expected an indented block (1780520270.py, line 13)

# Article baseclass definition

In [1]:
from pydantic.dataclasses import dataclass
from pydantic import HttpUrl
from datetime import datetime

@dataclass
class Article:
    title: str
    summary: str
    text:str
    publication_date: datetime
    source:str
    url: HttpUrl
    img_url: HttpUrl
    tags:list
    region:str
    language:str

In [2]:
import pandas as pd


In [ ]:
from GoogleNews import GoogleNews

# Translating the keywords into all languages

In [3]:
import asyncio
from async_google_trans_new import AsyncTranslator
from unsync import unsync
import aiohttp
import backoff


class KeywordsPerMarket:

    def __init__(self,input_keywords:list,region:str,language:str,input_language:str='en'):
        self.input_keywords = input_keywords
        self.region = region
        self.language = language
        self.market = f"{language}-{region}"
        self.input_language = input_language

    @property
    def keywords(self):
        
        @unsync
        @backoff.on_exception(backoff.expo, aiohttp.ClientError, max_time=60)
        async def translate_keywords(input_keywords:list,target_language:str,input_language:str):
            translator = AsyncTranslator(url_suffix='com')
            tasks = [translator.translate(keyword,lang_tgt=target_language,lang_src=input_language) for keyword in input_keywords]
            return await asyncio.gather(*tasks)

        return translate_keywords(self.input_keywords,self.language,self.input_language).result()


keyword = KeywordsPerMarket(['Hello world','evening'], region='nl',language='nl')
keyword.market
keyword.keywords

['Hallo Wereld ', 'avond ']

 Translate some keywords to multiple languages

In [4]:
from pydantic import BaseModel,ComputedField,computed_field
import asyncio
from async_google_trans_new import AsyncTranslator
from unsync import unsync
import aiohttp
import backoff

class KeywordsPerMarket(BaseModel):
    input_keywords:list 
    region:str 
    language:str
    market = ComputedField(lambda self:f"{self.language}-{self.region}")
    input_language:str = 'en'

    @computed_field
    @property
    def keywords(self):
        
        @unsync
        @backoff.on_exception(backoff.expo, aiohttp.ClientError, max_time=60)
        async def translate_keywords(input_keywords:list,target_language:str,input_language:str):
            translator = AsyncTranslator(url_suffix='com')
            tasks = [translator.translate(keyword,lang_tgt=target_language,lang_src=input_language) for keyword in input_keywords]
            return await asyncio.gather(*tasks)

        return translate_keywords(self.input_keywords,self.language,self.input_language).result()

In [5]:
keywords = ['aviation','airlines','flight','airplane']
languages = ['es','nl','de','fr']

keywords_per_market = [KeywordsPerMarket(input_keywords=keywords,language=language,region='') for language in languages]

In [6]:
keywords_per_market

[KeywordsPerMarket(input_keywords=['aviation', 'airlines', 'flight', 'airplane'], region='', language='es', input_language='en'),
 KeywordsPerMarket(input_keywords=['aviation', 'airlines', 'flight', 'airplane'], region='', language='nl', input_language='en'),
 KeywordsPerMarket(input_keywords=['aviation', 'airlines', 'flight', 'airplane'], region='', language='de', input_language='en'),
 KeywordsPerMarket(input_keywords=['aviation', 'airlines', 'flight', 'airplane'], region='', language='fr', input_language='en')]

In [66]:
KeywordsPerMarket(input_keywords=keywords,language='nl',region='').keywords


['luchtvaart ', 'luchtvaartmaatschappijen ', 'vlucht ', 'vliegtuig ']

In [ ]:
import pandas as pd 

# 
pd.DataFrame([{'keyword':kpm.keywords,'language':kpm.language} for kpm in keywords_per_market])

,keyword,language
0,"[aviación , aerolíneas , vuelo , avión ]",es
1,"[luchtvaart , luchtvaartmaatschappijen , vluch...",nl
2,"[Luftfahrt , Fluglinien , Flug , Flugzeug ]",de
3,"[aviation , compagnies aériennes , vol , avion ]",fr


In [57]:
from datetime import datetime
from pydantic.dataclasses import dataclass
from pydantic import BaseModel



class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime = None


user = User(id='42', signup_ts='2032-06-21T12:00')
print(user)

id=42 name='John Doe' signup_ts=datetime.datetime(2032, 6, 21, 12, 0)


In [59]:
pd.DataFrame([user,user])

,0,1,2
0,"(id, 42)","(name, John Doe)","(signup_ts, 2032-06-21 12:00:00)"
1,"(id, 42)","(name, John Doe)","(signup_ts, 2032-06-21 12:00:00)"


In [8]:
import pandas as pd
pd.DataFrame([KeywordsPerMarket(input_keywords=keywords,language=language,region='').dict() for language in languages])

TypeError: __init__() takes 1 positional argument but 2 were given